In [1]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import MCXGate, RXGate, RZZGate
from qiskit.circuit import Parameter
from sympy.physics.quantum import TensorProduct
from sympy import Matrix
import numpy as np

In [2]:
def adjacency_matrix(V, edges):
    # Initialize an empty V x V matrix with all zeros
    matrix = [[0] * V for _ in range(V)]
    
    # Populate the matrix based on the edges
    for edge in edges:
        u, v = edge
        matrix[u][v] = 1
        matrix[v][u] = 1  # Undirected graph
    
    return np.array(matrix)

In [4]:
'Distance Matrix'
D = np.array([[0.        ,2 , 3],
 [2, 0.        , 5 ],
 [3, 5 , 0.        ]])

'Number of Nodes'
V = 3
edges = [(0, 1), (1, 2), (2, 0)]

'Adjacency matrix'
A = adjacency_matrix(V, edges)

'Affinity matrix'
K = TensorProduct(A, D)
print(K)

[[0. 0. 0. 0. 2. 3. 0. 2. 3.]
 [0. 0. 0. 2. 0. 5. 2. 0. 5.]
 [0. 0. 0. 3. 5. 0. 3. 5. 0.]
 [0. 2. 3. 0. 0. 0. 0. 2. 3.]
 [2. 0. 5. 0. 0. 0. 2. 0. 5.]
 [3. 5. 0. 0. 0. 0. 3. 5. 0.]
 [0. 2. 3. 0. 2. 3. 0. 0. 0.]
 [2. 0. 5. 2. 0. 5. 0. 0. 0.]
 [3. 5. 0. 3. 5. 0. 0. 0. 0.]]
